# Laboratorio 6 - GPUs con Python
**alumno09:** Laura Llamas López

## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [4]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.13 ms ± 29.9 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.4 ms ± 599 ns per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.5 ms ± 53.8 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


## a) Librería CuPy: Aceleración con GPU

En esta sección utilizamos CuPy para acelerar el cálculo usando la GPU. CuPy es una librería muy similar a NumPy pero diseñada específicamente para GPUs. La mayoría de funciones de NumPy tienen el mismo nombre en CuPy.

Vamos a medir el tiempo en dos escenarios:
1. **Con copia de datos**: Creamos los arrays en CPU (NumPy) y los copiamos a la GPU
2. **Sin copia de datos**: Creamos los arrays directamente en la GPU usando CuPy

In [5]:
import cupy as cp
from cupyx.profiler import benchmark

# =============================================================================
# CASO 1: CON COPIA DE DATOS (CPU -> GPU)
# =============================================================================
print("\n" + "="*70)
print("CASO 1: Con copia de datos (CPU -> GPU)")
print("="*70)

# Función que incluye las copias CPU<->GPU
def grade2_with_copy(x_cpu, y_cpu, a, b, c):
    x_gpu = cp.asarray(x_cpu)  # copiar a GPU
    y_gpu = cp.asarray(y_cpu)  
    z_gpu = a * x_gpu**2 + b * y_gpu + c  # calcular en GPU
    z_cpu = cp.asnumpy(z_gpu)  # copiar resultado a CPU
    return z_cpu

# Medir tiempo con benchmark
exec_with_copy = benchmark(grade2_with_copy, (a_cpu, b_cpu, a, b, c), 
                           n_repeat=10, n_warmup=2)
time_with_copy = np.average(exec_with_copy.gpu_times) * 1e3
print(f"Tiempo con copia: {time_with_copy:.3f} ms")

# =============================================================================
# CASO 2: SIN COPIA DE DATOS (creación directa en GPU)
# =============================================================================
print("\n" + "="*70)
print("CASO 2: Sin copia de datos (arrays creados directamente en GPU)")
print("="*70)

# Crear arrays directamente en GPU
x_gpu = cp.random.rand(size)
y_gpu = cp.random.rand(size)

# Calcular usando la ufunc ya definida (funciona con CuPy arrays)
z_gpu = grade2_ufunc(x_gpu, y_gpu, a, b, c)

# Medir tiempo con benchmark (sin copias)
exec_no_copy = benchmark(grade2_ufunc, (x_gpu, y_gpu, a, b, c), 
                         n_repeat=10, n_warmup=2)
time_no_copy = np.average(exec_no_copy.gpu_times) * 1e3
print(f"Tiempo sin copia: {time_no_copy:.3f} ms")

# =============================================================================
# RESUMEN
# =============================================================================
print("\n" + "="*70)
print("RESUMEN - CuPy con GPU")
print("="*70)
print(f"Con copia de datos: {time_with_copy:.3f} ms")
print(f"Sin copia de datos: {time_no_copy:.3f} ms")
print(f"Overhead de copia:  {time_with_copy - time_no_copy:.3f} ms")
print(f"Factor: {time_with_copy / time_no_copy:.2f}x más lento con copias")


CASO 1: Con copia de datos (CPU -> GPU)
Tiempo con copia: 20.703 ms

CASO 2: Sin copia de datos (arrays creados directamente en GPU)
Tiempo sin copia: 4.415 ms

RESUMEN - CuPy con GPU
Con copia de datos: 20.703 ms
Sin copia de datos: 4.415 ms
Overhead de copia:  16.287 ms
Factor: 4.69x más lento con copias


## b) Librería Numba: Creación de ufunc para GPU

En esta sección utilizamos Numba con el decorador `@vectorize` para crear una ufunc que se ejecuta directamente en la GPU. A diferencia de CuPy que es una librería completa, Numba nos permite convertir funciones Python en ufuncs optimizadas para GPU.

El decorador `@vectorize` con `target='cuda'` genera automáticamente código CUDA que se ejecuta en paralelo en la GPU. No necesitamos paralelizar explícitamente porque la GPU ya maneja la paralelización internamente.

Mediremos el tiempo en dos escenarios:
1. **Con copia automática**: Pasamos arrays de NumPy directamente y Numba gestiona las copias CPU↔GPU automáticamente
2. **Sin copia**: Copiamos manualmente los arrays a GPU antes de medir para excluir el overhead de transferencia

In [6]:
from numba import vectorize
from numba import cuda
from cupyx.profiler import benchmark
import time

# Definir funciones de timing para Numba GPU
def timing_numba_GPU(func, args, n_repeat=10, n_warmup=1):
    """Timing function based on numba cuda.synchronize()"""
    for i in range(n_warmup):
        out = func(*args)
    cuda.synchronize()
    timing = np.empty(n_repeat)
    for i in range(timing.size):
        tic = time.time()
        out = func(*args)
        cuda.synchronize()
        toc = time.time()
        timing[i] = toc - tic
    return timing.mean()*1e3

def timing_numba_GPU2(func, args, n_repeat=10, n_warmup=1):
    """Timing function based on cupy Events"""
    import cupy as cp
    gpu_start = cp.cuda.Event()
    gpu_end = cp.cuda.Event()
    for i in range(n_warmup):
        out = func(*args)
    gpu_start.record()
    for i in range(n_repeat):
        out = func(*args)
    gpu_end.record()
    gpu_end.synchronize()
    t_gpu = cp.cuda.get_elapsed_time(gpu_start, gpu_end)
    return t_gpu / n_repeat

# Crear ufunc con @vectorize para GPU
@vectorize(['float64(float64, float64, float64, float64, float64)'], target='cuda')
def grade2_numba_gpu(x, y, a, b, c):
    return a * x**2 + b * y + c

# =============================================================================
# CASO 1: CON COPIA AUTOMÁTICA (CPU -> GPU automático)
# =============================================================================
print("\n" + "="*70)
print("CASO 1: Con copia automática (Numba gestiona CPU <-> GPU)")
print("="*70)

# Método 1: benchmark de CuPy
exec_benchmark = benchmark(grade2_numba_gpu, (a_cpu, b_cpu, a, b, c), 
                          n_repeat=10, n_warmup=2)
time_benchmark = np.average(exec_benchmark.gpu_times) * 1e3
print(f"Benchmark (CuPy):     {time_benchmark:.3f} ms")

# Método 2: timing_numba_GPU (cuda.synchronize)
time_numba1 = timing_numba_GPU(grade2_numba_gpu, (a_cpu, b_cpu, a, b, c), 
                               n_repeat=10, n_warmup=2)
print(f"timing_numba_GPU:     {time_numba1:.3f} ms")

# Método 3: timing_numba_GPU2 (CuPy Events)
time_numba2 = timing_numba_GPU2(grade2_numba_gpu, (a_cpu, b_cpu, a, b, c), 
                                n_repeat=10, n_warmup=2)
print(f"timing_numba_GPU2:    {time_numba2:.3f} ms")

# =============================================================================
# CASO 2: SIN COPIA (arrays ya en GPU)
# =============================================================================
print("\n" + "="*70)
print("CASO 2: Sin copia (arrays copiados manualmente a GPU)")
print("="*70)

# Copiar arrays a GPU manualmente ANTES de medir (usando CuPy como en apartado a)
x_gpu_numba = cp.asarray(a_cpu)
y_gpu_numba = cp.asarray(b_cpu)

# Método 1: benchmark de CuPy
exec_benchmark_no_copy = benchmark(grade2_numba_gpu, (x_gpu_numba, y_gpu_numba, a, b, c), 
                                   n_repeat=10, n_warmup=2)
time_benchmark_no_copy = np.average(exec_benchmark_no_copy.gpu_times) * 1e3
print(f"Benchmark (CuPy):     {time_benchmark_no_copy:.3f} ms")

# Método 2: timing_numba_GPU (cuda.synchronize)
time_numba1_no_copy = timing_numba_GPU(grade2_numba_gpu, (x_gpu_numba, y_gpu_numba, a, b, c), 
                                       n_repeat=10, n_warmup=2)
print(f"timing_numba_GPU:     {time_numba1_no_copy:.3f} ms")

# Método 3: timing_numba_GPU2 (CuPy Events)
time_numba2_no_copy = timing_numba_GPU2(grade2_numba_gpu, (x_gpu_numba, y_gpu_numba, a, b, c), 
                                        n_repeat=10, n_warmup=2)
print(f"timing_numba_GPU2:    {time_numba2_no_copy:.3f} ms")

# =============================================================================
# RESUMEN
# =============================================================================
print("\n" + "="*70)
print("RESUMEN - Numba con GPU (promedio de los 3 métodos)")
print("="*70)

# Calcular promedio de los 3 métodos
time_with_copy_avg = (time_benchmark + time_numba1 + time_numba2) / 3
time_no_copy_avg = (time_benchmark_no_copy + time_numba1_no_copy + time_numba2_no_copy) / 3

print(f"Con copia automática: {time_with_copy_avg:.3f} ms")
print(f"Sin copia:            {time_no_copy_avg:.3f} ms")
print(f"Overhead de copia:    {time_with_copy_avg - time_no_copy_avg:.3f} ms")
print(f"Factor: {time_with_copy_avg / time_no_copy_avg:.2f}x más lento con copias")


CASO 1: Con copia automática (Numba gestiona CPU <-> GPU)
Benchmark (CuPy):     18.471 ms
timing_numba_GPU:     18.871 ms
timing_numba_GPU2:    18.159 ms

CASO 2: Sin copia (arrays copiados manualmente a GPU)
Benchmark (CuPy):     2.189 ms
timing_numba_GPU:     2.093 ms
timing_numba_GPU2:    1.985 ms

RESUMEN - Numba con GPU (promedio de los 3 métodos)
Con copia automática: 18.500 ms
Sin copia:            2.089 ms
Overhead de copia:    16.411 ms
Factor: 8.86x más lento con copias


## c) Interpretación de resultados

#### Tabla 1: Resultados en CPU (baseline)

| Método | Tiempo | Observaciones |
|--------|--------|---------------|
| **Numba (@njit)** | 8.13 ms | Mejor rendimiento en CPU |
| **NumPy ufunc** | 18.4 ms | 2.26x más lento que Numba |

#### Tabla 2: Resultados en GPU con CuPy

| Escenario | Tiempo | Speedup vs Numba CPU | Overhead de copia |
|-----------|--------|----------------------|-------------------|
| **Con copia de datos** | 20.70 ms | 0.39x (2.55x más lento) | 16.29 ms |
| **Sin copia de datos** | 4.42 ms | **1.84x más rápido** | - |

#### Tabla 3: Resultados en GPU con Numba @vectorize

| Escenario | Tiempo | Speedup vs Numba CPU | Overhead de copia |
|-----------|--------|----------------------|-------------------|
| **Con copia automática** | 18.50 ms | 0.44x (2.28x más lento) | 16.41 ms |
| **Sin copia** | 2.09 ms | **3.89x más rápido** | - |

### Análisis de resultados

Los experimentos muestran diferencias importantes entre las dos librerías de GPU y el impacto crítico de las transferencias de memoria.

Cuando comparamos **CuPy** y **Numba** sin copias de datos, Numba obtiene tiempos notablemente mejores (2.09 ms frente a 4.42 ms de CuPy), logrando una aceleración de 3.89x respecto a Numba CPU. Esto probablemente se debe a que Numba genera código CUDA específico para la función mediante el decorador @vectorize, mientras que CuPy ejecuta operaciones más genéricas sobre arrays en GPU. CuPy, aunque más lento que Numba, sigue siendo competitivo con un speedup de 1.84x.

El **overhead de las transferencias CPU↔GPU** es prácticamente idéntico en ambas librerías (16.29 ms para CuPy y 16.41 ms para Numba), lo cual tiene sentido porque ambas utilizan el mismo hardware y protocolo de comunicación. Este overhead domina completamente el tiempo de ejecución cuando incluimos las copias: tanto CuPy (20.70 ms) como Numba (18.50 ms) resultan más lentas que NumPy en CPU (18.4 ms), y mucho más lentas que Numba CPU (8.13 ms).

En conclusión, usar la GPU solo tiene sentido cuando los datos ya están en memoria GPU o cuando realizamos suficientes operaciones consecutivas para amortizar el costo de transferencia. Para un único cálculo vectorial como el de este ejercicio, Numba CPU (@njit) sigue siendo competitiva por su simplicidad y buen rendimiento. Sin embargo, si trabajamos con pipelines de múltiples operaciones sobre los mismos datos, Numba GPU sería la opción más rápida, seguida de CuPy GPU.